In [60]:
%cd "/content/drive/MyDrive/Colab Notebooks/DACON_PLANT"

/content/drive/MyDrive/Colab Notebooks/DACON_PLANT


In [ ]:
import numpy as np
import random
import os
import math
import argparse
import time
import matplotlib.pyplot as plt

from glob import glob
import pandas as pd
import cv2 as cv
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import torchvision.models as models
from torchvision import transforms

# image segmentation

In [26]:
def segmentation(): # plan segmentation function
    root = "dataset/test/image"
    for path in tqdm(os.listdir(root)):
        sub_path = root + '/' + path

        img = cv.imread(sub_path)
        hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

        # 초록색 rgb를 range로 설정하고, 다른 rgb값들을 masking
        mask = cv.inRange(hsv, (36, 0, 0), (100, 200,200))
        imask = mask>0
        green = np.zeros_like(img, np.uint8)
        green[imask] = img[imask]

        # 이미지를 흑백 컬러로 전환
        gray = cv.cvtColor(green, cv.COLOR_BGR2GRAY)
        ret, thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

        # segmentation폴더에 저장
        ret_path = "image_segmentation"
        cv.imwrite(ret_path + "/segmentation/" + path, thresh)


In [27]:
segmentation()

  0%|          | 0/460 [00:00<?, ?it/s]

In [ ]:
root = 'image_segmentation/segmentation'
for path in os.listdir(root):
        print(path[-8:-4])

In [38]:
def count_pixel():
    root = 'image_segmentation/segmentation'
    area_list = []
    file_name = []
    cnt = 0
    for path in tqdm(os.listdir(root)):
        seg_path = root + "/" + path

        img = cv.imread(seg_path)

        sought = [255,255,255]
        white  = np.count_nonzero(np.all(img==sought,axis=2))
        sought = [0,0,0]
        black  = np.count_nonzero(np.all(img==sought,axis=2))
        total = white + black
        area = black/total 

        dst = path[-8:-4]

        # print(f"filename: {dst}")
        # print(f"white: {white}")
        # print(f"black: {black}")
        # print(f"area of plant leaf: {area}")
            
        file_name.append(dst)
        area_list.append(area)

        cnt += 1
        if cnt % 100 == 0:
            print(cnt)
            
        
    with open('leaf_area.csv', 'w') as f:
        for i in range(len(file_name)):
            f.write("{0} ".format(file_name[i]))
            f.write("{0:0.3f} \n".format(area_list[i]))

In [39]:
count_pixel()

  0%|          | 0/460 [00:00<?, ?it/s]

100
200
300
400
